# Project 3


## Importing Modules and staring local db engine

In [ ]:
#Importing modules

import boto3

from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from database_utils import DatabaseConnector
import importlib
import json
import numpy as np
import pandas as pd
import re
import requests
from sqlalchemy import create_engine
from sqlalchemy import inspect
import tabula
import yaml


#starting local database

#read creds
uploader = DatabaseConnector()
ml_yaml = 'ml_dbs.yaml'
uploader.read_db_creds(ml_yaml)

# initialising and returning an sqlalchemy database engine.

uploading = uploader.init_db_engine_postgresql()



## RDS ETL: User


In [ ]:
### 1. Getting data from the RDS database

## read credentials
connector_rds = DatabaseConnector()
ai_core_yaml = 'db_creds.yaml'
connector_rds.read_db_creds(ai_core_yaml)

## initialising and returning an sqlalchemy database engine.
connection_rds = connector_rds.init_db_engine()

## Reading the data from the RDS database
connector_tables_rds = connector_rds.list_db_tables()


### 2. Extracting the 'legacy_users' table to a pandas DataFrame.
user_table_rds = 'legacy_users'
extractor_rds = DataExtractor(connection_rds)
user_df_rds = extractor_rds.read_rds_table(user_table_rds)
print(user_df_rds.head(5))
print(user_df_rds.info())

### 3.Cleaning data
clean_data_rds = DataCleaning()
clean_df_rds = clean_data_rds.clean_user_data(user_df_rds)

### 4. Loading the data
#load clean rds dataframe into local database with a new table
uploader.upload_to_db(uploading, clean_df_rds, 'dim_users')

## PDF ETL: Card Details

In [ ]:
### 1.Getting the data from PDF & load in df
connector_pdf = DatabaseConnector()
extractor_pdf = DataExtractor(connector_pdf)

link = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf"
extraction_pdf = extractor_pdf.retrieve_pdf_data(link)
print(extraction_pdf.head(5))
print(extraction_pdf.info())

### 2.Clean the pdf df

clean_pdf = DataCleaning()

clean_df_pdf =clean_pdf.clean_card_data(extraction_pdf)

### 3. load into db

uploader.upload_to_db(uploading, clean_df_pdf, 'dim_card_details')

## API ETL:  Store







In [ ]:
### 1. Getting Data from API


headers = {
    'x-api-key': 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'
}

number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'

connector_api = DatabaseConnector()

extractor_api = DataExtractor(connector_api)

number_of_stores = extractor_api.list_number_of_stores(number_of_stores_endpoint, headers)
print(f"Number of stores: {number_of_stores}")

### 2. Extracting the table to a pandas DataFrame
retrieve_store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/2'

stores_df = extractor_api.retrieve_stores_data(retrieve_store_endpoint, headers, 2)

### 3.Cleaning data

clean_stores_api = DataCleaning()
clean_storesdf_api = clean_stores_api.called_clean_store_data(stores_df)

### 4. Loading the data
uploader.upload_to_db(uploading, clean_storesdf_api, 'dim_store_details')

## S3 ETL: Products


In [ ]:
### 1. Getting data

connector_s3 = DatabaseConnector()
extractor_s3 = DataExtractor(connector_s3)


### 2. Putting into pd df

bucket = 'data-handling-public'
object_key = "products.csv"
pathway = '/Users/student/AICORE/AWS/Project_3/products.csv'

extraction_s3 = extractor_s3.download_csv_from_s3(bucket, object_key, pathway)
print(extraction_s3.head(20))
print(extraction_s3.info())

### 3. Cleaning data

clean_s3 = DataCleaning()

clean_df_s3 = clean_s3.called_clean_store_data(extraction_s3)


clean_df_s3 = clean_s3.called_clean_store_data(extraction_s3)


converted_df_s3 = clean_s3.convert_product_weights(clean_df_s3)
print(converted_df_s3.head())


clean_df_s3 = clean_s3.clean_products_data(converted_df_s3)


### 4. Loading into local db
uploader.upload_to_db(uploading, clean_df_s3, 'dim_products')


## AWS DB ETL: Orders

In [ ]:
connector_tables_rds = connector_rds.list_db_tables()

#'orders_table'


In [ ]:
orders_table_rds = 'orders_table'

extractor_rds = DataExtractor(connection_rds)

orders_df_rds = extractor_rds.read_rds_table(orders_table_rds)
print(orders_df_rds.head(5))
print(orders_df_rds.info())

#cleaning the orders table

clean_ordersdf_rds = clean_data_rds.clean_user_data(orders_df_rds)

## dropping columns

ordersdf_rds = clean_ordersdf_rds.drop(['first_name','last_name','1' ], axis = 1)
print(ordersdf_rds.info())

#uploading the updated orders table into the local db
uploader.upload_to_db(uploading, ordersdf_rds, 'orders_table')


## JSON S3 ETL: Date Details

In [ ]:
#Preparing the extration

connector_json_s3 = DatabaseConnector()
extractor_json_s3 = DataExtractor(connector_json_s3)
bucket = 'data-handling-public'
object_key_json = "date_details.json"

#Get the object

extraction_json_s3 = extractor_json_s3.extract_json_from_s3(bucket, object_key_json)
print(extraction_json_s3.head(20))
print(extraction_json_s3.info())

#Clean the data
clean_json_s3 = DataCleaning()

clean_df_json_s3 = clean_json_s3.called_clean_store_data(extraction_json_s3)

#upload to local db
uploader.upload_to_db(uploading, clean_df_json_s3, 'dim_date_times')